In [ ]:
import mlrun
import mlrun.feature_store as fs
import pandas as pd
from mlrun.datastore.targets import ParquetTarget, NoSqlTarget, CSVTarget

In [ ]:
project_name = "feature-sets"
project = mlrun.get_or_create_project(project_name, context="./")

## Creating y fs

In [ ]:
y = 20
for i in range(1, y):
    feature_set = fs.FeatureSet(
        f"cancer_{i}",
        f"{i} helps identify abnormal cell growth",
        entities=[fs.Entity("label")],
    )
    feature_set.metadata.labels = {
        "dataset-type": "training",
        "model-version": "v1.2",
        "source": "sensor-a",
        "environment": "production",
        "owner": "team-x",
    }
    targets = [ParquetTarget(partitioned=False), NoSqlTarget(), CSVTarget()]
    features = pd.read_csv("./cancer.csv")
    feature_set["mean_area"] = fs.Feature(
        description="Average area of the object", labels={"unit": "square centimeters"}
    )
    ingest_df = feature_set.ingest(
        features, targets, infer_options=fs.InferOptions.default()
    )

## Creating fs with x tags

In [ ]:
x = 15
for i in range(x):
    feature_set = fs.FeatureSet("cancer_1", entities=[fs.Entity("label")])
    feature_set.save(tag="tag" + str(i), versioned=True)

## Creating y fv

In [ ]:
y = 20
for i in range(1, y):
    features = [f"cancer_{i}.*"]

    vector = fs.FeatureVector(
        f"vector_{i}",
        features=features,
        description=f"based on corresponding {i} feature set",
    )
    vector.metadata.labels = {"diagnosis": "malignant", "sample-origin": "biopsy"}
    resp = vector.get_offline_features()

## Creating fv with z tags

In [ ]:
z = 15
for i in range(z):
    features = ["cancer_1.*"]

    vector = fs.FeatureVector(f"vector_0", features=features)
    vector.save(tag="tag" + str(i), versioned=True)
    resp = vector.get_offline_features()